In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import librosa
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from algorithms import *
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import time
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('dsl_data/development.csv')
df_eval = pd.read_csv('dsl_data/evaluation.csv')

In [3]:
# condition = (df['Current language used for work/school'] == np.unique(df_eval['Current language used for work/school'].values)[0]) & (df['First Language spoken'] == np.unique(df_eval['First Language spoken'].values)[0]) & (df['Self-reported fluency level '] == np.unique(df_eval['Self-reported fluency level '].values)[0])
# df = df.loc[condition]

In [4]:
cols = ['Id','Self-reported fluency level ', 'First Language spoken', 'Current language used for work/school']
df.drop(columns=cols,inplace=True)
df_eval.drop(columns=cols,inplace=True)

In [5]:
df['intent'] = df['action'] + df['object']
dict = {}
for i, el in enumerate(df['intent'].unique()):
    dict[el] = i

df['intent'] = df['intent'].apply(lambda x: dict[x])

In [6]:
cols = ['gender', 'ageRange']
one_hot_encode_columns(df,cols)
one_hot_encode_columns(df_eval,cols)

,path,speakerId,male,41-65
0,dsl_data/audio/speakers/NgQEvO2x7Vh3xy2xz/f53c...,NgQEvO2x7Vh3xy2xz,1,0
1,dsl_data/audio/speakers/k5bqyxx2lzIbrlg9/1d5f8...,k5bqyxx2lzIbrlg9,1,0
2,dsl_data/audio/speakers/7B4XmNppyrCK977p/1c0d5...,7B4XmNppyrCK977p,1,1
3,dsl_data/audio/speakers/k5bqyxx2lzIbrlg9/275c3...,k5bqyxx2lzIbrlg9,1,0
4,dsl_data/audio/speakers/V4ZbwLm9G5irobWn/b7c7a...,V4ZbwLm9G5irobWn,0,1
...,...,...,...,...
1450,dsl_data/audio/speakers/oOK5kxoW7dskMbaK/02f5d...,oOK5kxoW7dskMbaK,0,0
1451,dsl_data/audio/speakers/oOK5kxoW7dskMbaK/87191...,oOK5kxoW7dskMbaK,0,0
1452,dsl_data/audio/speakers/ppymZZDb2Bf4NQnE/f2a9e...,ppymZZDb2Bf4NQnE,0,1
1453,dsl_data/audio/speakers/aokxBz9LxXHzZzay/f347b...,aokxBz9LxXHzZzay,0,0


In [7]:
audio_feature_extraction(df)
audio_feature_extraction(df_eval)

In [8]:
def trim_audio(row):
    data = row['data']
    trimmed_data, index = librosa.effects.trim(data, top_db=20, frame_length=512, hop_length=64)
    return trimmed_data

df['data'] = df.apply(trim_audio, axis=1)
df_eval['data'] = df_eval.apply(trim_audio, axis=1)

df['data'] = df['data'].apply(lambda x: np.hstack([x[part[0]:part[-1]] for part in librosa.effects.split(x, top_db=30)]))
df_eval['data'] = df_eval['data'].apply(lambda x: np.hstack([x[part[0]:part[-1]] for part in librosa.effects.split(x, top_db=30)]))
def pad_audio(row):
    audio = row['data']
    sr = row['rate']
    max_length = int(30 * sr) # 30 seconds * sampling rate
    return np.pad(audio, (0, max_length - len(audio)), mode='constant', constant_values=0)

df['data'] = df.apply(pad_audio, axis=1)
df_eval['data'] = df_eval.apply(pad_audio, axis=1)

In [9]:
df

,path,speakerId,action,object,gender,ageRange,intent,data,rate
0,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/0a312...,2BqVo8kVB2Skwgyb,change language,none,female,22-40,0,"[-0.004102989, 0.0017110795, 0.0046248036, -0....",22050
1,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/0ee42...,2BqVo8kVB2Skwgyb,activate,music,female,22-40,1,"[-0.0034481208, -0.0040454636, -0.0046446496, ...",22050
2,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/1d9f3...,2BqVo8kVB2Skwgyb,deactivate,lights,female,22-40,2,"[0.0014983321, 0.0014417854, 0.0013267061, 0.0...",22050
3,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/269fc...,2BqVo8kVB2Skwgyb,increase,volume,female,22-40,3,"[-0.0074363947, -0.007070067, -0.005553694, -0...",22050
4,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/5bbda...,2BqVo8kVB2Skwgyb,increase,volume,female,22-40,3,"[-0.00081694237, -0.0005666799, -0.00014418755...",22050
...,...,...,...,...,...,...,...,...,...
9849,dsl_data/audio/speakers/vnljypgejkINbBAY/4fb3d...,vnljypgejkINbBAY,decrease,volume,male,22-40,4,"[0.0013669498, 0.0024509765, 0.0036058861, 0.0...",22050
9850,dsl_data/audio/speakers/vnljypgejkINbBAY/59e6a...,vnljypgejkINbBAY,deactivate,lights,male,22-40,2,"[-0.001144933, -0.013419567, 0.005848511, 0.01...",22050
9851,dsl_data/audio/speakers/vnljypgejkINbBAY/5c81c...,vnljypgejkINbBAY,deactivate,lights,male,22-40,2,"[-0.0013577347, 0.015522823, 0.023896402, 0.01...",22050
9852,dsl_data/audio/speakers/vnljypgejkINbBAY/5ef42...,vnljypgejkINbBAY,deactivate,lights,male,22-40,2,"[-0.0047846185, -0.0029887287, -0.0042724935, ...",22050


In [10]:
y = df['intent']

In [11]:
new_columns = ['action', 'object', 'intent']
df.drop(columns = new_columns, inplace=True)

In [12]:
def extract_spectrogram(dataframe):
    audio_paths = dataframe["path"].values
    spectrograms = []
    for data in dataframe['data']:

        # Compute the spectrogram
        spectrogram = np.abs(librosa.stft(data))

        # Split the spectrogram into 20 chunks
        n_chunks = 20
        chunks = np.array_split(spectrogram, n_chunks)

        # Append the chunks to the list of spectrograms
ograms.append(chunks)

    # Add the spectrogram chunks to the dataframe as new columns
    for i in range(n_chunks):
        dataframe["mean_spectrogram_" + str(i)] = [np.mean(chunk[i]) for chunk in spectrograms]
        dataframe["std_spectrogram_" + str(i)] = [np.std(chunk[i]) for chunk in spectrograms]
    return dataframe

In [13]:
df = extract_spectrogram(df)
# df_eval = extract_spectrogram(df_eval)

: 

: 

In [ ]:
def extract_chroma(dataframe):
    audio_paths = dataframe["path"].values
    chromas = []
    for data in dataframe['data']:
        # Compute the chroma feature
        chroma = librosa.feature.chroma_stft(data)

        # Split the chroma into 20 chunks
        n_chunks = 20
        chunks = np.array_split(chroma, n_chunks)

        # Append the chunks to the list of chromas
        chromas.append(chunks)

    # Add the chroma chunks to the dataframe as new columns
    for i in range(n_chunks):
        dataframe["mean_chroma_" + str(i)] = [np.mean(chunk[i]) for chunk in chromas]
        dataframe["std_chroma_" + str(i)] = [np.std(chunk[i]) for chunk in chromas]
    return dataframe

In [ ]:
df = extract_chroma(df)
df_eval = extract_chroma(df_eval)

In [ ]:
def extract_tonnetz(dataframe):
    audio_paths = dataframe["path"].values
    tonnetzs = []
    for data, rate in zip(df['data'], df['rate']):

        # Compute the tonnetz
        tonnetz = librosa.feature.tonnetz(y=data, sr=rate)

        # Split the tonnetz into 20 chunks
        n_chunks = 20
        chunks = np.array_split(tonnetz, n_chunks)

        # Append the chunks to the list of tonnetzs
        tonnetzs.append(chunks)

    # Add the tonnetz chunks to the dataframe as new columns
    for i in range(n_chunks):
        dataframe["mean_tonnetz_" + str(i)] = [np.mean(chunk[i]) for chunk in tonnetzs]
        dataframe["std_tonnetz_" + str(i)] = [np.std(chunk[i]) for chunk in tonnetzs]
    return dataframe


In [ ]:
def extract_spectral_contrast(dataframe):
    audio_paths = dataframe["path"].values
    spectral_contrasts = []
    for data in dataframe['data']:
        # Compute the spectral contrast
        spectral_contrast = librosa.feature.spectral_contrast(data)

        # Split the spectral contrast into 20 chunks
        n_chunks = 20
        chunks = np.array_split(spectral_contrast, n_chunks)

        # Append the chunks to the list of spectral_contrasts
        spectral_contrasts.append(chunks)
    for i in range(n_chunks):
        dataframe["mean_spectral_contrast_" + str(i)] = [np.mean(chunk[i]) for chunk in spectral_contrasts]
        dataframe["std_spectral_contrast_" + str(i)] = [np.std(chunk[i]) for chunk in spectral_contrasts]
    return dataframe

In [ ]:
df = extract_spectral_contrast(df)
df_eval = extract_spectral_contrast(df_eval)

In [ ]:
def extract_spectral_contrast(dataframe):
    audio_paths = dataframe["path"].values
    spectral_contrasts = []
    for data in dataframe['data']:

        # Compute the spectral contrast
        spectral_contrast = librosa.feature.spectral_contrast(data)

        # Split the spectral contrast into 20 chunks
        n_chunks = 20
        chunks = np.array_split(spectral_contrast, n_chunks)

        # Append the chunks to the list of spectral contrasts
        spectral_contrasts.append(chunks)

    # Add the spectral contrast chunks to the dataframe as new columns
    for i in range(n_chunks):
        dataframe["mean_spectral_contrast_" + str(i)] = [np.mean(chunk[i]) for chunk in spectral_contrasts]
        dataframe["std_spectral_contrast_" + str(i)] = [np.std(chunk[i]) for chunk in spectral_contrasts]
    return dataframe

In [ ]:
df = extract_spectral_contrast(df)
df_eval = extract_spectral_contrast(df_eval)

In [ ]:
def extract_rms(dataframe):
    audio_paths = dataframe["path"].values
    rms = []
    for data in dataframe['data']:
        # Compute the RMS
        rms_data = librosa.feature.rms(data)[0]
        # Split the RMS into 20 chunks
        n_chunks = 20
        chunks = np.array_split(rms_data, n_chunks)
        # Append the chunks to the list of RMS
        rms.append(chunks)
    # Add the RMS chunks to the dataframe as new columns
    for i in range(n_chunks):
        dataframe["mean_rms_" + str(i)] = [np.mean(chunk[i]) for chunk in rms]
        dataframe["std_rms_" + str(i)] = [np.std(chunk[i]) for chunk in rms]
    return dataframe

In [ ]:
df = extract_rms(df)
df_eval = extract_rms(df_eval)

In [ ]:
def extract_mfcc(dataframe):
    audio_paths = dataframe["path"].values
    mfccs = []
    for data in dataframe['data']:
            # Compute the MFCCs
        mfcc = librosa.feature.mfcc(data)

        # Split the MFCCs into 20 chunks
        n_chunks = 20
        chunks = np.array_split(mfcc, n_chunks, axis = 1)

        # Append the chunks to the list of MFCCs
        mfccs.append(chunks)

    # Add the MFCC chunks to the dataframe as new columns
    for i in range(n_chunks):
        dataframe["mean_mfcc_" + str(i)] = [np.mean(chunk[i]) for chunk in mfccs]
        dataframe["std_mfcc_" + str(i)] = [np.std(chunk[i]) for chunk in mfccs]
    return dataframe


In [ ]:
df = extract_mfcc(df)
df_eval = extract_mfcc(df_eval)

In [ ]:
# Iterate over each column in the DataFrame
for col in df.columns:
    # Check if the column contains 1D or 2D arrays
    if isinstance(df[col].values[0], (list, np.ndarray)) and len(df[col].values[0]) > 1:
        # Drop the column if it contains 1D or 2D arrays
        df.drop(col, axis=1, inplace=True)
for col in df_eval.columns:
    # Check if the column contains 1D or 2D arrays
    if isinstance(df_eval[col].values[0], (list, np.ndarray)) and len(df_eval[col].values[0]) > 1:
        # Drop the column if it contains 1D or 2D arrays
        df_eval.drop(col, axis=1, inplace=True)

In [ ]:
def drop_nan_columns(df):
    df.dropna(axis=1, how='any', inplace=True)
    return df
drop_nan_columns(df)
drop_nan_columns(df_eval)

In [ ]:
X = df.copy()
X_eval = df_eval.copy()

In [ ]:
# X = pd.read_csv('new_df.csv')
# X_eval = pd.read_csv('new_df_eval.csv')

In [ ]:
X.drop(columns=['path','speakerId','rate'],inplace=True)
X_eval.drop(columns=['path','speakerId','rate'],inplace=True)

In [ ]:
X

In [ ]:
# X =  normalize_dataframe(X)
# X_eval = normalize_dataframe(X_eval)
def standardize(x_train, x_test):
    mean = np.mean(x_train, axis=0)
    std = np.std(x_train, axis=0)
    x_train_standardized = (x_train - mean) / std
    x_test_standardized = (x_test - mean) / std
    return x_train_standardized, x_test_standardized
X, X_eval = standardize(X, X_eval)

In [ ]:
# top_n_features = select_top_n_features(X, y)

In [ ]:
def new_svm_model(X_train, y_train, X_test):
    clf = SVC(kernel = 'rbf', C=10, gamma='auto')
    # train the model on the training data
    clf.fit(X_train, y_train)
    # predict the target values for the test data
    # returning the y_predict
    return clf.predict(X_test)
y_pred = new_svm_model(X, y, X_eval)

In [ ]:
key_mapping = np.vectorize(lambda x: next(key for key, value in dict.items() if value == x))
keys_arr = key_mapping(y_pred)

In [ ]:
y_evaluation_df = pd.DataFrame(keys_arr, columns = ['Predicted'])
y_evaluation_df.index.name = 'Id'

In [ ]:
from datetime import datetime
now = int(time.time())
readable_time = datetime.fromtimestamp(now).strftime('%H:%M:%S')
y_evaluation_df.to_csv(f'evaluation/copy_predictions-{readable_time}.csv')